In [2]:
import sys
sys.path.append("../src/") 
import os

import model_manipulation as mm
import cobra
import cplex 
import libsbml
import pandas as pd
import copy
from pathlib import Path


In [3]:
#Read 2-cell model
model = cobra.io.read_sbml_model("../model/ios2164_1cell.xml")
#Estimate inf
inf = 1e9



Set parameter Username
Academic license - for non-commercial use only - expires 2022-04-03


In [4]:
#Solver
model.solver = 'cplex'

In [5]:
#Initialize medium so as to induce Photoautotrophic conditions (i.e. no Sucrose import)
model_media = mm.read_medium_csv('../misc/photo_medium.csv', model)
    
model_media['EX_no3(e)'] = inf
model_media['EX_nh4(e)'] = 0
model_media['EX_h2o(e)'] = inf

model_media['EX_h(e)'] = 0
model_media = model.medium

model.medium

{'EX_co2(e)': 1000000,
 'EX_h2o(e)': 1000000,
 'EX_no3(e)': 1000000,
 'EX_o2(e)': 1000000,
 'EX_pi(e)': 1000000,
 'EX_so4(e)': 1000000,
 'EX_so3(e)': 1000000,
 'EX_h2s(e)': 1000000,
 'EX_nh4(e)': 1000000,
 'EX_fe2(e)': 1000000,
 'EX_fe3(e)': 1000000,
 'EX_mg2(e)': 1000000,
 'EX_photonVis(e)': 1000000}

In [6]:
#This code block is unnecessary because this is a single-cell model

# # #Add constraints to model
# #This code block contains constraints that would simulate the assimilation rates of bs and m cells in a two-cell system (such as those seen near the midvein region of rice leaves)
# # #BS photon flux must be the same/less than M flux (Adapted from B&B, 2019)
# # photon_import = model.reactions.get_by_id("EX_photonVis(e)")




# co2tex_m = model.reactions.CO2tex_M
# co2tex_bs = model.reactions.CO2tex_BS

# #CO2 Uptake must be constrained to A (Net assimilation rate) which is around 29 +- 1.2 umol CO2 m-2 s-1 for wild type rice
# #Lower bound based  on Dark CO2 uptake rates from Ermakova et al (2020)
# #CO2 BS intake must be limited owing to its position in the leaf tissue

# # # #CO2 intake needs to be mostly in the M cells. Not sure of the exact value:
# # # Values from Von Caemmerer (2000) in intermediate C3-C4 photosynthetic plants is 1 mmol CO2 m-2 s-1 per 20 CO2 m-2 s-1, which puts it at 20:1 ratio
# # # #try checking Danila et al. (2016) on ratio of surface area of M cell to BS cell
# # # #I'll assume that the ratio is at 20:1 in the meantime, meaning that there is minimal gas exchange into the BS cell that would equate to around ~1 umol CO2 m-2s-1
# co2_ratio_cons = mm.set_fix_flux_ratio({co2tex_m.id:20, co2tex_bs.id:1},model)
# model.add_cons_vars(co2_ratio_cons)

# # #Automatic na palang nalalagay siya as constraint

# # #First is the BS and M Cell constraints
# # #What if I restrict yung import ng O2?

# # #Let's try restricting O2 to the value from Lakshmanan et al. (2016) = 3.312 mmol O2 m-2 s-1
# # #Actually this value is also reflected in Von Caemmerer's "Modelling C3 metabolism" (pg. 85)
# o2tex_bs = model.reactions.O2tex_BS

# bs_o2_cons = model.problem.Constraint(0.047*co2tex_bs.flux_expression 
#                                       - 1 * o2tex_bs.flux_expression, lb=0, ub=1000)

# model.add_cons_vars(bs_o2_cons)



In [10]:
# #This code block contains constraints specific for enzyme rate constraints
#This approach is derived from Bogart & Myers (2016) where they constrained the enzyme rate 
#fluxes in each of the 2-cell segments to a specific upper bound while keeping the lower bound
#At 0.

# #PEPC
wt_pepc = 2.4 #umol m-2 s-1
wt_mdh = 11.18 #umol m-2 s-1
wt_nadp_me = 0.14 #umol m-2 s-1
wt_ppdk = 0.31 #umol m-2 s-1
wt_CA = 7.5 #umol m-2  s-1 bar-1 (Constrained to CO2 amounting to 400-500 mbar) (1 bar = 15.74 umol m-2 s-1)

#PEPC constraint (Reaction id: PPCc)
pepc = model.reactions.PPCc

wt_pepc_cons = model.problem.Constraint(pepc.flux_expression, 
                                        lb = 0, ub = wt_pepc)
model.add_cons_vars(wt_pepc_cons)

#PPDK constraints (Reaction id: PPDKS) (note that this is found in the chloroplast?) 
#Not detected via immunolocalization but enzyme activity is detected

ppdks = model.reactions.PPDKs

wt_ppdks_cons = model.problem.Constraint(ppdks.flux_expression, 
                                         lb = 0, ub = wt_ppdk)
model.add_cons_vars(wt_ppdks_cons)

#Malate Dehydrogenase 
mdh2c = model.reactions.MDH2c
mdh2s = model.reactions.MDH2s

wt_mdh_cons = model.problem.Constraint(mdh2c.flux_expression  
                                       + mdh2s.flux_expression, 
                                       lb= 0, ub=wt_mdh)
model.add_cons_vars(wt_mdh_cons)


#NADP-ME (Since no signal is detected in WT, no locational constraints are imposed)
#Let's see if I can force it to have a small amount of flux 
nadp_me = model.reactions.MDH2s

wt_nadpme_cons = model.problem.Constraint(nadp_me.flux_expression,
                                         lb= 0, ub=wt_nadp_me)
model.add_cons_vars(wt_nadpme_cons)
#Weird, it causes the activity of RBCL to significantly vary from the actual values


#I should add constraints for Carbonic Anhydrase
#I should constrain it to 0.4 ubar, which would constitute ambient CO2 partial pressure
#
#HCO3Es
#HCO3Ec
#HCO3Em
#Flux is reversible so constraints are bi-directional


hco3es = model.reactions.HCO3Es.flux_expression
hco3ec = model.reactions.HCO3Ec.flux_expression
hco3em = model.reactions.HCO3Em.flux_expression


ca_cons = model.problem.Constraint(hco3es + hco3ec + hco3em,
                                  lb = -wt_CA, ub = wt_CA)
model.add_cons_vars(ca_cons)
#Rbcl constaints

#Let's try to add Rubisco flux ratio

rbpc = model.reactions.RBPCs
rbpo = model.reactions.RBPOs

#Constraint such that it is limited to 132 umol m-2 s-1
rbcl_vcmax_cons = model.problem.Constraint(rbpc.flux_expression, lb = 0, ub= 132)

model.add_cons_vars(rbcl_vcmax_cons)

#Constraints for rbcl flux

rbcl_vcvo = model.problem.Constraint(3*(rbpo.flux_expression) 
                                     - 1*(rbpc.flux_expression),
                                     lb=0,ub=1000)

model.add_cons_vars(rbcl_vcvo)

#What if I simply constrained that of the M cell one to 3:1?
#This constraint is pretty good actually. 
#This allows the system to be set at a specific Vc/Vo rate while still allowing local variation 
#wherein Rubisco may act in an uncoupled fashion and may have favorable internal vc/vo rates.


In [36]:
# Let's try restricting O2 intake again to a specific amount, let's see if PPFD responds accordingly

with model:
    
    model_media['EX_photonVis(e)'] = 1000
    model_media['EX_co2(e)'] = 29
    model_media['EX_o2(e)'] = 2.2618 #2.2618
    model.medium = model_media
#Constrain O2 intake to 3.312 mmol O2 gcw -1 d-1 ~~ 2.2618 umol O2 m-2 s-1 (from Lakshmanan et al. 2016)
    #Change objective function to reflect Mature leaf
    model.reactions.get_by_id('Straw_Biomass').objective_coefficient = 0
    model.reactions.get_by_id('DM_Phloem').objective_coefficient = 1
    sample_fluxes = cobra.flux_analysis.pfba(model)
    output = model.summary(fva=0.95)
output

Metabolite,Reaction,Flux,Range,C-Number,C-Flux
co2_e,EX_co2(e),15.21,[9.618; 29],1,100.00%
h2o_e,EX_h2o(e),26.39,[-19.38; 71.43],0,0.00%
no3_e,EX_no3(e),1.616,[0; 8.671],0,0.00%
o2_e,EX_o2(e),2.262,[-2.736; 2.262],0,0.00%
photonVis_e,EX_photonVis(e),1000,[948.9; 1000],0,0.00%
Metabolite,Reaction,Flux,Range,C-Number,C-Flux
ltlnnp_c,DM<NNP(c),0,[-0.09823; 0],27,0.00%
3-non_s,DM_3-NON(s),0,[-0.1536; 0],9,0.00%
3-nond_s,DM_3-NOND(s),0,[-0.1577; 0],9,0.00%
9-nonan_s,DM_9-NONAN(s),0,[-0.1577; 0],9,0.00%


In [ ]:
model.reactions.

In [35]:
## This is cool, the M cell is similar to the WT rate of 83.59+-5.63 :)
print('rbcl M cell: ', sample_fluxes['RBPCs'], 'rbcl BS cell: ',sample_fluxes['RBPCs'])
print('rbcl M cell (photorespiration)', sample_fluxes['RBPOs'], 'rbcl BS cell (PR)', sample_fluxes['RBPOs'])
print('vc/vo M:', sample_fluxes['RBPCs']/sample_fluxes['RBPOs'], 'vc/vo BS:', sample_fluxes['RBPCs']/sample_fluxes['RBPOs'])
print('PEPC M', sample_fluxes['PPCc'], 'PEPC M', sample_fluxes['PPCc'])
print('NADP-ME M', sample_fluxes['MDH2s'], 'NADP-ME BS', sample_fluxes['MDH2s'])
print('Biomass M: ', sample_fluxes['Straw_Biomass'], 'Biomass BS', sample_fluxes['Straw_Biomass'])
print('Phloem M: ', sample_fluxes['DM_Phloem'], 'Phloem BS', sample_fluxes['DM_Phloem'])
print('co2 consumption BS', sample_fluxes['CO2tex'])
print('o2 consumption M', sample_fluxes['O2tex'], 'o2 consumption BS', sample_fluxes['O2tex'])
print('Photosystem II M', sample_fluxes['PSIINC'], 'PSII BS', sample_fluxes['PSIINC'])
print('PSI M', sample_fluxes['PSIMR'], 'PSI BS', sample_fluxes['PSIMR'])
print('PPFD M: ', sample_fluxes['PRISM_white_LED'], 'PPFD BS: ', sample_fluxes['PRISM_white_LED'])

pd_rxn = [x for x in model.reactions if "pd" in x.id and "h2o" not in x.id]
pd_abs_flux = 0
for pds in pd_rxn:
    pd_abs_flux += abs(sample_fluxes[pds.id])
    
print('pd_abs_flux: ', pd_abs_flux)

rbcl M cell:  11.017887926993456 rbcl BS cell:  11.017887926993456
rbcl M cell (photorespiration) 3.672629308997819 rbcl BS cell (PR) 3.672629308997819
vc/vo M: 2.9999999999999996 vc/vo BS: 2.9999999999999996
PEPC M 1.199729475652319 PEPC M 1.199729475652319
NADP-ME M 0.0 NADP-ME BS 0.0
Biomass M:  0.0 Biomass BS 0.0
Phloem M:  0.8996171833025808 Phloem BS 0.8996171833025808
co2 consumption BS 15.211818557406211
o2 consumption M 2.2618 o2 consumption BS 2.2618
Photosystem II M 100.92341514071941 PSII BS 100.92341514071941
PSI M 136.05966479603956 PSI BS 136.05966479603956
PPFD M:  1000.0 PPFD BS:  1000.0
pd_abs_flux:  0


In [ ]:
for rxns in model.reactions:
    if "pd" in rxn.id

In [35]:
model.reactions.RBPCs_BS.summary()

In [41]:

ppfd_solns_2cell = pd.DataFrame(index=list(i.id for i in model.reactions))

#Iterate from range 0 to 2000 w/ increments of 50 (max ppfd)
with model:
    for i in range(0,2050,50):
        ppfd = i 
        #Set medium to change photon flux and re-add to model
        model_media['EX_photonVis(e)'] = ppfd
        model_media['EX_co2(e)'] = 29
        model_media['EX_o2(e)'] = 3
        model.medium = model_media
        #         model_media['EX_photonVis(e)'] = ppfd
        #         print('current PPFD:', model_media['EX_photonVis(e)'])
        #         model.medium = model_media
        #         print('current PPFD bounds', model.reactions.get_by_id('EX_photonVis(e)').bounds)
        
        
        #Change objective function to reflect Mature leaf
        model.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 1
        model.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 1
        model.reactions.get_by_id('DM_Phloem_M').objective_coefficient = 0
        model.reactions.get_by_id('DM_Phloem_BS').objective_coefficient = 0


        #perform pFBA & convert to dataframe
        tc_soln = cobra.flux_analysis.pfba(model).to_frame()
        tc_soln = pd.DataFrame({ppfd:tc_soln['fluxes']}, index=list(tc_soln.index))
        print("PPFD:", ppfd, " RBPC_M cell flux:", tc_soln[ppfd]['RBPCs_M'])
        #Append to Dataframe
        ppfd_solns_2cell[ppfd] = tc_soln

#Code now works

PPFD: 0  RBPC_M cell flux: 0.787848502645273
PPFD: 50  RBPC_M cell flux: 4.029041627223497
PPFD: 100  RBPC_M cell flux: 7.193769338355178
PPFD: 150  RBPC_M cell flux: 10.122697829148118
PPFD: 200  RBPC_M cell flux: 13.087722195449178
PPFD: 250  RBPC_M cell flux: 15.7987969759423
PPFD: 300  RBPC_M cell flux: 18.399152346350334
PPFD: 350  RBPC_M cell flux: 20.4293179172801
PPFD: 400  RBPC_M cell flux: 21.846103408720378
PPFD: 450  RBPC_M cell flux: 23.697405172057877
PPFD: 500  RBPC_M cell flux: 26.355147468911277
PPFD: 550  RBPC_M cell flux: 28.321414766940407
PPFD: 600  RBPC_M cell flux: 29.374914257242317
PPFD: 650  RBPC_M cell flux: 32.26926745333002
PPFD: 700  RBPC_M cell flux: 34.25473823123802
PPFD: 750  RBPC_M cell flux: 36.22963592186014
PPFD: 800  RBPC_M cell flux: 38.20453361238255
PPFD: 850  RBPC_M cell flux: 40.16497282607887
PPFD: 900  RBPC_M cell flux: 40.66813053046557
PPFD: 950  RBPC_M cell flux: 42.55033324275232
PPFD: 1000  RBPC_M cell flux: 46.10412437464492
PPFD: 105

In [ ]:
model.medium

In [ ]:
model.reactions.RBPCs_BS

In [ ]:
for rxns in model.reactions:
    print(rxns.id, sample_fluxes[rxns.id])

In [ ]:
#Note:
#Unconstrained CO2 intake leads to a saturated  net CO2 assimilation rate of around 40 umol m-2 s-1
#Better to constrain this to known uptake rates instead


In [ ]:
#Note:
#If I restrict the flux of H2O to the BS cell only what happens is that CO2 is transported higher to the M cell rather than 

In [ ]:
#Oh wtf it works, below 500 ppfd no net intake below dark respiration is observed tapos nasasaturate siya agad to  the max value
#Cooooool. Based on Weber et al (1986) the critical point is at around 500-600 umol he m-2 s-1
#Weber, J. A., Tenhunen, J. D., Gates, D. M., & Lange, O. L. (1987). Effect of photosynthetic photon flux density on carboxylation efficiency. Plant physiology, 85(1), 109-114.


In [ ]:
#What if I limited influx of CO2 from the M cell to the BS cell?

In [ ]:
#What I've done was unconstrained CO2 exchange to allow for CO2 efflux. At around 300-400 ppfd a net release of CO2 is observed from the mdoel, consistent 
#with Nobel (2020)'s  'Leaves and Fluxes'
#I need to read more on this. This is actually quite interesting
# 

In [ ]:
#Constraining O2 consumption to around 3.312 forces water to be consumed by the system, unlike when O2 is unconstrained.
#However, the same behavior when it comes to CO2 assimilation still is being produced.
#Constraining O2 consumption also forces Nitrate consumption to be constrained to realistic levels (when compared to non-constrained O2 where consumption was 10-fold )
#I suspect that this is definitely due to how PPFD flux is structured. 
#A cop-out to this would be to double PPFD per run and measuring PPFD flux in the specific reactions instead.

In [ ]:
#I can probably do the benchmarking right now. The model behaves quite interestingly actually

In [ ]:
#Note: Turning off other prism reactions (red, green, blue, etc) while retaining only white light reactions lowers net carbon assimilation
#I think this is in consideration on how white light is partitioned to several wavelengths, which prevents it from being fully utilized.

In [ ]:
#Other artifacts as of the moment include the following:
#model doesn't produce any Oxygen. Maybe I should add a specific demand reaction for it?
#Based on FVA, the model now produces some oxygen via the demand reaction. 

In [ ]:
#Model also facilitates transfer of metabolites unlike before.

In [ ]:
#Questions:
#Should I restrict Nutrient flow to the Bundle Sheath Cell only or should I allow flux to both the M and BS cell?
#Apparently kasi dito only the Bundle sheath cells produce any biomass
#This is to ensure that only CO2 and Light flux are the only limiting constraints to the system.